In [1]:
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support.select import Select

In [26]:
path = Service("/Users/shin/chromedriver")
driver = webdriver.Chrome(service=path) #크롬 브라우저 사용

df = pd.DataFrame()

URL = "https://job.incruit.com/jobdb_list/searchjob.asp?ct=1&ty=1&cd=150&page="
for i in range(1, 210):
    
    driver.get(URL+str(i))
    driver.implicitly_wait(time_to_wait=60)

    div = driver.find_elements(By.CLASS_NAME, "accent")
    for item in div:
        tmp = pd.DataFrame([item.find_element(By.TAG_NAME, "a").get_attribute('href')])
        df = pd.concat([df, tmp])
    time.sleep(1)
    
driver.close()
df

,0
0,https://job.incruit.com/jobdb_list/searchjob.a...
0,https://job.incruit.com/jobdb_list/searchjob.a...
0,https://job.incruit.com/jobdb_list/searchjob.a...
0,https://job.incruit.com/jobdb_list/searchjob.a...
0,https://job.incruit.com/jobdb_info/jobpost.asp...
...,...
0,https://job.incruit.com/jobdb_info/jobpost.asp...
0,https://job.incruit.com/jobdb_info/jobpost.asp...
0,https://job.incruit.com/jobdb_info/jobpost.asp...
0,https://job.incruit.com/jobdb_info/jobpost.asp...


In [29]:
df.drop_duplicates([0]).to_csv("incruit_link.csv")

In [3]:
link_df = pd.read_csv("incruit_link.csv").drop(['Unnamed: 0'], axis=1)
link_df

,0
0,https://job.incruit.com/jobdb_info/jobpost.asp...
1,https://job.incruit.com/jobdb_info/jobpost.asp...
2,https://job.incruit.com/jobdb_info/jobpost.asp...
3,https://job.incruit.com/jobdb_info/jobpost.asp...
4,https://job.incruit.com/jobdb_info/jobpost.asp...
...,...
12447,https://job.incruit.com/jobdb_info/jobpost.asp...
12448,https://job.incruit.com/jobdb_info/jobpost.asp...
12449,https://job.incruit.com/jobdb_info/jobpost.asp...
12450,https://job.incruit.com/jobdb_info/jobpost.asp...


In [4]:
from tqdm import tqdm

path = Service("/Users/shin/chromedriver")
driver = webdriver.Chrome(service=path)

df = pd.DataFrame()
for URL in tqdm(link_df['0']):
    
    driver.get(URL)
    driver.implicitly_wait(time_to_wait=60)
    try:
        content = driver.find_element(By.TAG_NAME, "iframe")
        driver.switch_to.frame(content)
        div = driver.find_element(By.ID, "content_job")
        tmp = pd.DataFrame([div.text])
        df = pd.concat([df, tmp])
        time.sleep(0.5)
    except:
        continue
    
driver.close()

df

100%|██████████████████████████████████| 12452/12452 [11:11:50<00:00,  3.24s/it]


,0
0,
0,1. 채용인원 및 부문\n□ 채용직급 / 인원 : 일반직 5급 / 28명 * 3개월...
0,행사 개요\n행 사 명 : MZ세대 맞춤형 글로벌 기업 취업특강\n일 정 : 202...
0,모집부문 및 자격요건\n❑ 모집인원 : 0명\n직무\n구분\n전공\n상세직무\n우대...
0,"본 입사지원 홈페이지는 크롬에 최적화되어 있으므로, 원활한 입사 지원을 위해 크롬 ..."
...,...
0,"[직무내용]\n담당업무\nㆍ웹, 모바일 서비스 기획\nㆍ서비스 요구사항 구체화 및 ..."
0,[직무내용]\n채용직무 : 앱 · 웹 개발 / 서비스\n- 담당업무 : 앱 · 웹 ...
0,"[직무내용]\n(주)바생 사진 & 영상 촬영 및 제작, 편집 정규직 채용합니다.\n..."
0,[직무내용]\n[함께 성장하는 회사 디비오피에서 안드로이드 앱 개발자를 모집합니다]...


In [11]:
df.to_csv("incruit_crawling_data.csv")